In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
# import mlflow
# import os
# from getpass import getpass

# os.environ['MLFLOW_TRACKING_USERNAME'] = input('Enter your DAGsHub username: ')
# os.environ['MLFLOW_TRACKING_PASSWORD'] = getpass('Enter your DAGsHub access token: ')
# os.environ['MLFLOW_TRACKING_PROJECTNAME'] = input('Enter your DAGsHub project name: ')

# mlflow.set_tracking_uri(f'https://dagshub.com/' + os.environ['MLFLOW_TRACKING_USERNAME'] 
#                         + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + '.mlflow')

# with mlflow.start_run(run_name="MLflow on Colab"):
#   mlflow.log_metric("m1", 2.0)
#   mlflow.log_param("p1", "mlflow-colab")

In [18]:
import wave
import os
import speech_recognition as sr
import librosa
import csv
import pandas as pd
import numpy as np
import torch
import torchaudio
import math, random
from torchaudio import transforms
import torch.nn.functional as F
from torch.nn import init
from torch.utils.data import random_split

In [29]:
Path_to_train = "/home/david/Elvis/MLOps_Strategist/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav"
Path_to_test = "/home/david/Elvis/MLOps_Strategist/ALFFA_PUBLIC/ASR/SWAHILI/data/test/wav5"
Path_to_train_labels = "/home/david/Elvis/MLOps_Strategist/ALFFA_PUBLIC/ASR/SWAHILI/data/train/text"
Path_to_test_labels="/home/david/Elvis/MLOps_Strategist/ALFFA_PUBLIC/ASR/SWAHILI/data/test/text"
Path_to_spectrograms = "/spectrograms"

In [30]:
Path_to_train_labels

'/home/david/Elvis/MLOps_Strategist/ALFFA_PUBLIC/ASR/SWAHILI/data/train/text'

In [31]:
#extract all .wav from subdirectories
def get_file_paths(dirname):
    file_paths = []  
    for root, directories, files in os.walk(dirname):
        for filename in files:
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  
    return file_paths

In [32]:
get_file_paths(Path_to_spectrograms)

[]

In [34]:
#create a folder with all the train files
import shutil
in_path = Path_to_train
out_path = "/all_files"
 
for dirpath, dirnames, filenames in os.walk(in_path):
    for filename in filenames:
        if filename.endswith(".wav"):
             src = os.path.join(dirpath, filename)
             dest = os.path.join(out_path, filename)
             shutil.copy2(src, dest)

FileNotFoundError: [Errno 2] No such file or directory: '/all_files/SWH-05-20110114_16k-emission_swahili_05h30_-_06h00_tu_20110114_part91.wav'

In [11]:
#Process the data
# def process_file(file):
#     r = sr.Recognizer()
#     a = ''
#     with sr.AudioFile(file) as source:
#         audio = r.record(source)    
#         try:
#             a =  r.recognize_google(audio)        
#         except sr.UnknownValueError:
#             a = "Google Speech Recognition could not understand audio"
#         except sr.RequestError as e:
#             a = "Could not request results from Google Speech Recognition service; {0}".format(e)  
#     return a

In [12]:
# def main():
#     files = get_file_paths(Path_to_train)                 # get all file-paths of all files in dirname and subdirectories
#     for file in files:                              # execute for each file
#         (filepath, ext) = os.path.splitext(file)    # get the file extension
#         file_name = os.path.basename(file)          # get the basename for writing to output file
#         if ext == '.wav':                           # only interested if extension is '.wav'
#             a = process_file(file)                  # result is returned to a
#             with open(OUTPUTFILE, 'a') as f:        # write results to file
#                 writer = csv.writer(f)
#                 writer.writerow(['file_name','google'])
#                 writer.writerow([file_name, a])

In [13]:
# #put extracted .wav files into a dataframe
# speech_df = pd.DataFrame (get_file_paths(Path_to_train),columns=['recording'])
# speech_df.head()

In [14]:
# #extract content of text file(labels)
# text_labels=pd.DataFrame(pd.read_csv(Path_to_train_labels, sep=',',header=None, names=['file_path'])['file_path'].str.split('\t',1).tolist(),
#                                  columns = ['file_path','text'])
# text_labels.head()

In [35]:
pd.set_option('display.max_colwidth',None)

In [36]:
#merge dataframes based on the name of the file path use metadata

#read text from every transcription audio
def read_text( text_path):
    text = []
    with open(text_path) as fp:
        line = fp.readline()
        while line:
            text.append(line)
            line = fp.readline()
    return text

In [37]:
label=[]
transcriptions = []
for t in read_text(Path_to_train_labels):
    name=t.split("\t")[1]
    # name=name.replace('(', '')
    # name=name.replace(')', '')
    name=name.replace('\n','')
    # name=name.replace(' ','')
    text=t.split("\t")[0]
    # text=text.replace("\t","")
    # name_to_text[name]=text
    label.append(text)
    transcriptions.append(name)

In [38]:
#get audio path , every path must corespond to transcription , get the transprion in the doc and append to audio path 
audio_path=[0]*len(transcriptions)
for d in get_file_paths(Path_to_spectrograms):
    _d = d.strip(".wav")
    sp = _d.split("/")[9]
    index = label.index(sp)
    audio_path[index] = d

In [39]:
#calculate duration 
duration_of_recordings=[]
for d in audio_path:
    audio, fs = librosa.load(d, sr=None)
    duration_of_recordings.append(float(len(audio)/fs))

RuntimeError: Error opening 0: File contains data in an unknown format.

In [ ]:
speech_data=pd.DataFrame({'key': audio_path,'text': transcriptions})
speech_data.head()

In [ ]:
recognizer = sr.Recognizer()

In [ ]:
# set energy threshold
# audios that are below this threshold will be considered silent
recognizer.energy_threshold = 300

In [18]:
# used to load audio file
#specifying sample rate will resize all the files i.e Audio will be automatically resampled to the given rate
class Loader:
  def __init__(self, sample_rate,duration,mono):
    self.sample_rate=sample_rate
    self.duration=duration
    self.mono=mono
    self.channel = 2
    self.shift_limit = 0.4
    self.n_mels=64 
    self.n_fft=1024 
    self.hop_len=None
    self.max_mask_pct=0.1 
    self.n_freq_masks=1 
    self.n_time_masks=1
    

  # def load_signal(self, filepath):
  #   signal=librosa.load(filepath,
  #                       sr=None,
  #                       duration=self.duration,
  #                       mono=self.mono)[0]    #librosa returns 2D array (signal,sample_rate) pick the signal here
  #   return signal

  # def load_sample_rate(self,filepath):
  #   self.sample_rate=librosa.load(filepath,
  #                       sr =None,
  #                       duration=self.duration,
  #                       mono=self.mono)[1]    #librosa returns 2D array (signal,sample_rate) pick the sample rate here
  #   return self.sample_rate

  # def load(self,filepath):     #get both signal and sample rate in one
  #   aud=librosa.load(filepath,
  #                       sr=None,
  #                       duration=self.duration,
  #                       mono=self.mono)
  #   return aud

  def load(self,filepath):
    sig, sr = torchaudio.load(filepath)
    aud = sig, sr
    return aud

  def rechannel(self, aud):    #convert mono to stereo
    # aud=self.aud
    sig, sr = aud
  

    if (sig.shape[0] == self.channel):
      # Nothing to do
      return self.aud

    if (self.channel == 1):
      # Convert from stereo to mono by selecting only the first channel
      resig = sig[:1, :]
    else:
      # Convert from mono to stereo by duplicating the first channel
      resig = torch.cat([sig, sig])

    aud = resig, sr

    return aud

  def resample(self,aud):                    #standardize sample rate
    sig, sr = aud
    
    if (sr == self.sample_rate):
      # Nothing to do
      return aud

    num_channels = sig.shape[0]
    # Resample first channel
    resig = torchaudio.transforms.Resample(sr, self.sample_rate)(sig[:1,:])
    if (num_channels > 1):
      # Resample the second channel and merge both channels
      retwo = torchaudio.transforms.Resample(sr, self.sample_rate)(sig[1:,:])
      resig = torch.cat([resig, retwo])
      aud = resig, self.sample_rate
    return aud

  # ----------------------------
  # Pad (or truncate) the signal to a fixed length 'max_ms' in milliseconds
  # ----------------------------
  def pad_trunc(self,aud):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * self.duration

    if (sig_len > max_len):
      # Truncate the signal to the given length
      sig = sig[:,:max_len]

    elif (sig_len < max_len):
      # Length of padding to add at the beginning and end of the signal
      pad_begin_len = random.randint(0, max_len - sig_len)
      pad_end_len = max_len - sig_len - pad_begin_len

      # Pad with 0s
      pad_begin = torch.zeros((num_rows, pad_begin_len))
      pad_end = torch.zeros((num_rows, pad_end_len))

      sig = torch.cat((pad_begin, sig, pad_end), 1)
      aud = sig, sr
    return aud

# ----------------------------
  # Shifts the signal to the left or right by some percent. Values at the end
  # are 'wrapped around' to the start of the transformed signal.
  # ----------------------------
  def time_shift(self,aud):
    sig,sr = aud
    _, sig_len = sig.shape
    shift_amt = int(random.random() * self.shift_limit * sig_len)
    return (sig.roll(shift_amt), sr)

  # ----------------------------
  # Generate a MelSpectrogram
  # ----------------------------
  def spectro_gram(self, aud):
    sig,sr = aud
    top_db = 80

    # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
    spec = transforms.MelSpectrogram(sr, n_fft=self.n_fft, hop_length=self.hop_len, n_mels=self.n_mels)(sig)

    # Convert to decibels
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)


    # ----------------------------
  # Augment the Spectrogram by masking out some sections of it in both the frequency
  # dimension (ie. horizontal bars) and the time dimension (vertical bars) to prevent
  # overfitting and to help the model generalise better. The masked sections are
  # replaced with the mean value.
  # ----------------------------
  def spectro_augment(spec):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec

    freq_mask_param = self.max_mask_pct * n_mels
    for _ in range(self.n_freq_masks):
      aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

    time_mask_param = self.max_mask_pct * n_steps
    for _ in range(self.n_time_masks):
      aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

    return aug_spec

In [19]:
# #resizes audios to have same length
# class Padder:
#   def __init__(self,mode="constant"):
#     self.mode=mode
#   def left_pad(self,array,num_missing_items):
#     padded_array=np.pad(array,
#                         (num_missing_items, 0),
#                         mode=self.mode)
#     return padded_array
#   def right_pad(self,array,num_missing_items):
#     padded_array=np.pad(array,
#                         (0,num_missing_items),
#                         mode=self.mode)
#     return padded_array

In [20]:
#Save spectrograms in a folder
class Saver:
  def __init__(self,feature_save_dir):
    self.feature_save_dir=feature_save_dir

  def feature_save(self,spec,filepath):
    save_path=self._generate_save_path(filepath)
    np.save(save_path, spec)

  def _generate_save_path(self, filepath):
    file_name = os.path.split(filepath)[1]
    save_path = os.path.join(self.feature_save_dir, file_name)
    return save_path

In [21]:
#call functions in Loader and Saver
class PreprocessingPipeline:
  '''Processes audio files in a directory by applying the following steps
    1. Load the data, convert to mono and resample sampling rate
    2. Pad the audio
  '''
  def __init__(self):
    # self.padder=None
    self.saver=None
    self._loader=None
    # self._num_expected_samples=None
   

 
  # def loader(self):
  #   return self._loader


 
  # def loader(self,loader):
  #   self.loader=loader
  #   self._num_expected_samples=int(loader.sample_rate*loader.duration)


  def process(self,audio_files_directory):
    for root, directories, files in os.walk(audio_files_directory):
        for filename in files:
            filepath = os.path.join(root, filename)
            self._process_file(filepath)
            print(f"Processed file {filepath}")
    
  def _process_file(self,filepath):
    signal=self.loader.load(filepath)
    signal = self.loader.rechannel(signal)
    signal = self.loader.resample(signal)
    # if self._is_padding_necessary(signal):
    #   signal=self._apply_padding(signal)
    signal = self.loader.pad_trunc(signal)
    signal = self.loader.time_shift(signal)
    spec = self.loader.spectro_gram(signal) 
    save_path=self.saver.feature_save(spec,filepath)

    

  # def _is_padding_necessary(self,signal):
  #   self.num_expected_samples=int(loader.sample_rate*loader.duration)
  #   if len(signal) < self.num_expected_samples:
  #     return True
  #   return False

  # def _apply_padding(self,signal):
  #   num_missing_samples=self.num_expected_samples - len(signal)
  #   padded_signal = self.padder.right_pad(signal, num_missing_samples)
  #   return padded_signal

In [22]:
DURATION=4000
SAMPLE_RATE=44100
MONO=False

In [23]:
loader=Loader(SAMPLE_RATE, DURATION, MONO)
# padder=Padder()
saver = Saver(Path_to_spectrograms)

NameError: name 'Path_to_spectrograms' is not defined

In [ ]:
preprocessing_pipeline=PreprocessingPipeline()
preprocessing_pipeline.loader=loader
# preprocessing_pipeline.padder=padder
preprocessing_pipeline.saver=saver

In [ ]:
preprocessing_pipeline.process(Path_to_train)

In [ ]:
char_map_str = """
 ' 0
 <SPACE> 1
 a 2
 b 3
 c 4
 d 5
 e 6
 f 7
 g 8
 h 9
 i 10
 j 11
 k 12
 l 13
 m 14
 n 15
 o 16
 p 17
 q 18
 r 19
 s 20
 t 21
 u 22
 v 23
 w 24
 x 25
 y 26
 z 27
 '' 28
 """

In [24]:
class TextTransform:
    """Maps characters to integers and vice versa"""
    def __init__(self,char_map_str):
        self.char_map_str = char_map_str
        self.char_map = {}
        self.index_map = {}
        for line in self.char_map_str.strip().split('\n'):
            ch, index = line.split()
            self.char_map[ch] = int(index)
            self.index_map[int(index)] = ch
        self.index_map[1] = ' '

    def text_to_int(self, text):
        """ Use a character map and convert text to an integer sequence """
        int_sequence = []
        for c in text:
            if c == ' ':
                ch = self.char_map['']
            else:
                ch = self.char_map[c]
            int_sequence.append(ch)
        return int_sequence

    def int_to_text(self, labels):
        """ Use a character map and convert integer labels to an text sequence """
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return ''.join(string).replace('', ' ')

text_transform = TextTransform(char_map_str)

NameError: name 'char_map_str' is not defined

In [ ]:
text_transform.text_to_int(speech_data['text'][2])
# speech_data.head()
# speech_data['text'][0]

In [ ]:
myds = speech_data

# Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

# Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)